### Libraries

In [7]:
import gym
import random
import numpy as np
from statistics import mean, median
from collections import Counter

## Declare important varibales

In [3]:
LR = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirment = 50
initial_games = 10000

## Launch simple game

In [6]:
def some_random_games_fisrt():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break

In [7]:
some_random_games_fisrt()

In [13]:
initial_population()

Average accepted score: 59.38235294117647
Median accepted score: 54.5
Counter({51.0: 6, 50.0: 4, 54.0: 3, 57.0: 2, 52.0: 2, 53.0: 2, 65.0: 2, 60.0: 1, 86.0: 1, 69.0: 1, 77.0: 1, 55.0: 1, 72.0: 1, 89.0: 1, 82.0: 1, 58.0: 1, 56.0: 1, 66.0: 1, 59.0: 1, 68.0: 1})


[[array([ 0.0109401 ,  0.21350712, -0.04717495, -0.33989373]), [1, 0]],
 [array([ 0.01521024,  0.01908704, -0.05397282, -0.06245236]), [1, 0]],
 [array([ 0.01559198, -0.17522119, -0.05522187,  0.21272527]), [0, 1]],
 [array([ 0.01208756,  0.02064502, -0.05096736, -0.09685318]), [1, 0]],
 [array([ 0.01250046, -0.17371081, -0.05290443,  0.17932407]), [0, 1]],
 [array([ 0.00902624,  0.02212672, -0.04931795, -0.12956788]), [0, 1]],
 [array([ 0.00946878,  0.2179192 , -0.0519093 , -0.43739309]), [1, 0]],
 [array([ 0.01382716,  0.02356896, -0.06065716, -0.16151491]), [1, 0]],
 [array([ 0.01429854, -0.17063453, -0.06388746,  0.11143233]), [1, 0]],
 [array([ 0.01088585, -0.3647856 , -0.06165882,  0.38329466]), [0, 1]],
 [array([ 0.00359014, -0.16884488, -0.05399292,  0.07182589]), [1, 0]],
 [array([ 2.13240985e-04, -3.63152864e-01, -5.25564054e-02,  3.46996921e-01]),
  [1, 0]],
 [array([-0.00704982, -0.5574894 , -0.04561647,  0.62265475]), [1, 0]],
 [array([-0.0181996 , -0.75194571, -0.03316337

In [12]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        if score >= score_requirment:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                
                training_data.append([data[0],  output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    
    print('Average accepted score:', mean(accepted_scores))
    print('Median accepted score:', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data


In [18]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR,
                        loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model

In [19]:
def train_model(training_data, model=False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data] 
    
    if not model:
        model = neural_network_model(input_size = len(X[0]))
        
    model.fit({'input':X}, {'targets':y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openaistuff')
    
    return model

In [24]:
training_data = initial_population()
model = train_model(training_data)

Average accepted score: 61.3235294117647
Median accepted score: 58.5
Counter({53.0: 3, 57.0: 3, 68.0: 3, 58.0: 3, 50.0: 3, 60.0: 2, 61.0: 2, 56.0: 2, 70.0: 2, 81.0: 1, 69.0: 1, 54.0: 1, 59.0: 1, 51.0: 1, 65.0: 1, 52.0: 1, 78.0: 1, 80.0: 1, 67.0: 1, 77.0: 1})


NameError: name 'input_data' is not defined

In [23]:
scores = []
choices = []

for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()
        if len(prev_obs) == 0:
            action = env.action_space.sample()
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
        choices.append(action)
        
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
    scores.apprend(score)
print('Average Score', sum(scores)/len(scores))
print('Choise 1: {}, Choice 2: {}'.format(choices.count(10/len(choices)),
                                         choices.count(0)/len(choices)))

NameError: name 'model' is not defined